In [ ]:
import numpy as np
from ddm import DDM
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.optimize import curve_fit
from tifffile import imread, imwrite

In [ ]:
# make sure images are in gray scale && dimensions aren't not too large (512x512 is ideal)
sampling_time = 1/14.88
ntau = 800
calibration = (325e-6)/(1280)

# list of files to analyze
samples = ["sample.tif"]

In [ ]:
# compute ddm
for i in range(len(samples)):
    print(f"[{i+1}/{len(samples)}]{samples[i]} : ")
    stack = imread(samples[i]).astype(np.double)
    ddmstack = DDM(stack, ntau, sampling_time)
    result = ddmstack.compute()
    imwrite( samples[i].split('.')[0]  + "_ddm.tif", result)

In [ ]:
# image structure function, to fit
def D(t, A, B, freq):
    return A * (1 - np.exp(-t*freq)) + B
D = np.vectorize(D)

ddms = ["sample_ddm.tiff"]
freqs = np.zeros(512**2)

In [ ]:
# azimuthal average
def radial_profile(data):
    y0,x0 = data.shape

    x,y = np.meshgrid(np.arange(data.shape[1]),np.arange(data.shape[0]))
    R = np.sqrt( (y-y0//2)**2 + (x-x0//2)**2)

    f = lambda r : data[(R >= r-.5) & (R < r+.5)].mean()
    maxr = data.shape[1] - x0//2
    r = np.linspace(1, maxr, num=maxr)

    mean = np.vectorize(f)(r)
    return r,mean

In [ ]:
qs = np.arange(10,60, 1) # pick some qs
radials = np.array([ [0] * ntau for _ in range(len(qs)) ])
for ddm in ["sample10_ddm.tif"]:
    stack = imread(ddm).astype(np.double)
    for t in range(ntau):
        _,mean = radial_profile(stack[t,:,:])
        for i in range(len(qs)):
            radials[i,t] = mean[qs[i]]

In [ ]:
delays = np.linspace(0, ntau*sampling_time, ntau)
for i in range(len(qs)):
    plt.plot(delays, radials[i,:]/np.max(radials[i,:]))

plt.xlabel(r"$\Delta t (s)$")
plt.ylabel(r"$D(q,\Delta t)/D_{\text{max}}$")
#plt.legend([f"q = {q}" for q in qs]);
plt.title("Forme du signal. On plot pour tous les $q$.");

In [ ]:
As = []
Bs = []
nus = []
for i in range(len(qs)):
    A,B,nu = curve_fit(D, delays, radials[i,:]/np.max(radials[i,:]), p0=[1,0,1])[0]
    As.append(A)
    Bs.append(B)
    nus.append(nu)

In [ ]:
ts = np.linspace(0, ntau*sampling_time, ntau)
plt.plot(ts, D(ts, A=As[2], B=Bs[2], freq=nus[2]), 'k');

In [ ]:
real_qs = qs * (2*np.pi)/(512*calibration)

In [ ]:
real_qs_squared = real_qs**2
test_nus = np.array(nus)

Dm, intercept, _, _, se = linregress(real_qs_squared, test_nus)
plt.plot(real_qs_squared, test_nus, 'o', color='k')

plt.xlabel(r"$q^2$")
plt.ylabel(r"$\nu$")

f = np.vectorize(lambda qsquared: Dm*qsquared + intercept)
plt.title(f"sample-lait4.tif, Dm = {Dm:.2e} $\pm$ {se:.2e}")
plt.plot(real_qs_squared, f(real_qs_squared), '--', color='k')
plt.grid()

In [ ]:
print(Dm)